In [57]:
import numpy as np
import pandas as pd
import os
import requests
from tqdm import tqdm
import time as tm
import optuna
from hdbscan.validity import validity_index
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm  
import warnings
import matplotlib.pyplot as plt
import hdbscan
from sklearn.metrics import silhouette_score
from sklearn.kernel_approximation import Nystroem
from sklearn.manifold import Isomap

In [58]:
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 

In [59]:
df2 = pd.read_csv('NeoModelos/minas_con_tiempos_puertos.csv')

In [60]:
df2 = df2[df2['Estado'] == 'ACTIVA']
df2 = df2[(df2['RecursoPrimarioInstalacion'] == 'COBRE') | (df2['RecursoMineroInstalacion'] == 'SALMUERA (LITIO)')] 


In [61]:
drop_cols = ['RutEmpresa','NombreEmpresa','RecursoMineroInstalacion','TipoInstalacion',
             'TipoRecursoInstalacion','RecursoPrimarioInstalacion', 'ComunaFaena', 
             'NombreFaena', 'CategoriaFaena', 'IdFaena', 'ProvinciaInstalacion', 
             'ComunaInstalacion','NombreInstalacion','IdTipoInstalacion','IdInstalacion',
             'Norte','Este','Huso','Datum','IdEstado','Estado']

# Also drop distance columns as requested
distance_cols = [col for col in df2.columns if col.startswith('dist_')]
all_cols_to_drop = drop_cols + distance_cols

df = df2.drop(columns=all_cols_to_drop)

print(f"Original shape: {df2.shape}")
print(f"After dropping columns: {df.shape}")
print(f"Remaining columns: {list(df.columns)}")

Original shape: (7930, 96)
After dropping columns: (7930, 60)
Remaining columns: ['RegionFaena', 'ProvinciaFaena', 'RegionInstalacion', 'Cota', 'Latitud', 'Longitud', 'time_Santiago_min', 'time_Valparaíso_min', 'time_Viña_del_Mar_min', 'time_Antofagasta_min', 'time_La_Serena_min', 'time_Coquimbo_min', 'time_Rancagua_min', 'time_Talca_min', 'time_Iquique_min', 'time_Arica_min', 'time_Calama_min', 'time_Copiapó_min', 'time_San_Bernardo_min', 'time_Curicó_min', 'time_Ovalle_min', 'Tiempo_Prt_Antofagasta', 'Tiempo_Prt_Bahia Agua Fresca', 'Tiempo_Prt_Bahia De Valdivia', 'Tiempo_Prt_Bahia De Valparaiso', 'Tiempo_Prt_Bahia Harris', 'Tiempo_Prt_Bahia Herradura Guayacan', 'Tiempo_Prt_Bahia Quintero (Ventanas)', 'Tiempo_Prt_Bahia San Vicente', 'Tiempo_Prt_Caleta Clarencia', 'Tiempo_Prt_Caleta Mina Elena', 'Tiempo_Prt_Caleta Patillos', 'Tiempo_Prt_Coquimbo', 'Tiempo_Prt_Coronel', 'Tiempo_Prt_Huasco', 'Tiempo_Prt_Iquique', 'Tiempo_Prt_Lirquen', 'Tiempo_Prt_Lota', 'Tiempo_Prt_Mejillones', 'Tiempo_P

In [62]:
import numpy as np
import pandas as pd
# --- CONFIGURACIÓN ---
# Umbral de corte. 0.95 es el estándar en la industria.
# Significa: "Si la columna A y B se parecen en un 95%, borra B".
CORRELATION_THRESHOLD = 0.999

print(f"--- Buscando Columnas Redundantes (Corr > {CORRELATION_THRESHOLD}) ---")

# 1. Calcular matriz de correlación (solo numéricas)
# Usamos el valor absoluto porque una correlación de -0.99 es tan redundante como 0.99
df_numeric = df.select_dtypes(include=[np.number])
corr_matrix = df_numeric.corr().abs()

# 2. Seleccionar el triángulo superior de la matriz
# (La matriz es simétrica, no queremos borrar ambas columnas, solo una)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 3. Identificar columnas a borrar
to_drop = [column for column in upper.columns if any(upper[column] > CORRELATION_THRESHOLD)]

if len(to_drop) > 0:
    print(f"⚠️ Se encontraron {len(to_drop)} columnas redundantes para eliminar:")
    # Imprimir las primeras 10 para no llenar la pantalla
    print(to_drop[:10], "..." if len(to_drop) > 10 else "")
    
    # 4. Eliminar del DataFrame original
    df = df.drop(columns=to_drop)
    print(f"✅ Columnas eliminadas. Nueva dimensión: {df.shape}")
else:
    print("✅ No se encontró redundancia excesiva.")

# Verificación
print(f"Columnas restantes: {df.shape[1]}")

--- Buscando Columnas Redundantes (Corr > 0.999) ---
⚠️ Se encontraron 37 columnas redundantes para eliminar:
['time_Viña_del_Mar_min', 'time_Coquimbo_min', 'time_Talca_min', 'time_San_Bernardo_min', 'time_Curicó_min', 'Tiempo_Prt_Antofagasta', 'Tiempo_Prt_Bahia De Valdivia', 'Tiempo_Prt_Bahia De Valparaiso', 'Tiempo_Prt_Bahia Harris', 'Tiempo_Prt_Bahia Herradura Guayacan'] ...
✅ Columnas eliminadas. Nueva dimensión: (7930, 23)
Columnas restantes: 23


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7930 entries, 0 to 15151
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   RegionFaena                   7930 non-null   object 
 1   ProvinciaFaena                7930 non-null   object 
 2   RegionInstalacion             7930 non-null   object 
 3   Cota                          7930 non-null   int64  
 4   Latitud                       7930 non-null   float64
 5   Longitud                      7930 non-null   float64
 6   time_Santiago_min             7928 non-null   float64
 7   time_Valparaíso_min           7928 non-null   float64
 8   time_Antofagasta_min          7930 non-null   float64
 9   time_La_Serena_min            7929 non-null   float64
 10  time_Rancagua_min             7928 non-null   float64
 11  time_Iquique_min              7928 non-null   float64
 12  time_Arica_min                7929 non-null   float64
 13  time_Ca

In [64]:

# ==========================================
# PREPARACIÓN DE DATOS (Corrección dtype=int)
# ==========================================

# 1. One-Hot Encoding con dtype=int explícito
# 'dtype=int' fuerza a que sean 1 y 0, no True/False.
df_encoded = pd.get_dummies(df, columns=['ProvinciaFaena'], drop_first=True, dtype=int)

# 2. Ahora sí, seleccionamos numéricas (los int pasarán el filtro)
df_model = df_encoded.select_dtypes(include=[np.number]).dropna()

# 3. Guardar nombres y crear matriz X
feature_names = df_model.columns.tolist()
X = df_model.values
valid_indices = df_model.index
print(f"✅ DataFrame corregido.")
print(f"   Filas: {X.shape[0]}")
print(f"   Columnas Totales: {len(feature_names)}")

# --- MAPEO DE ÍNDICES ---
geo_idxs = [i for i, col in enumerate(feature_names) if col in ['Latitud', 'Longitud']]
log_idxs = [i for i, col in enumerate(feature_names) if col.startswith('time_') or col.startswith('Tiempo_Prt_')]
# Ahora esto debería encontrar las columnas porque son 'int' y sobrevivieron el filtro
cat_idxs = [i for i, col in enumerate(feature_names) if col.startswith('ProvinciaFaena_')]
other_idxs = [i for i in range(len(feature_names)) if i not in geo_idxs + log_idxs + cat_idxs]

print(f"\nResumen de Variables:")
print(f"   - Geográficas: {len(geo_idxs)}")
print(f"   - Logísticas (Tiempos): {len(log_idxs)}")
print(f"   - Categóricas (Provincias): {len(cat_idxs)} ")
print(f"   - Otras: {len(other_idxs)}")



# 4. Escalado
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)


✅ DataFrame corregido.
   Filas: 7908
   Columnas Totales: 47

Resumen de Variables:
   - Geográficas: 2
   - Logísticas (Tiempos): 17
   - Categóricas (Provincias): 27 
   - Otras: 1


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7930 entries, 0 to 15151
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   RegionFaena                   7930 non-null   object 
 1   ProvinciaFaena                7930 non-null   object 
 2   RegionInstalacion             7930 non-null   object 
 3   Cota                          7930 non-null   int64  
 4   Latitud                       7930 non-null   float64
 5   Longitud                      7930 non-null   float64
 6   time_Santiago_min             7928 non-null   float64
 7   time_Valparaíso_min           7928 non-null   float64
 8   time_Antofagasta_min          7930 non-null   float64
 9   time_La_Serena_min            7929 non-null   float64
 10  time_Rancagua_min             7928 non-null   float64
 11  time_Iquique_min              7928 non-null   float64
 12  time_Arica_min                7929 non-null   float64
 13  time_Ca

In [66]:
# Silenciamos la advertencia específica de compatibilidad sklearn/hdbscan
warnings.filterwarnings("ignore", category=FutureWarning, message=".*force_all_finite.*")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*ensure_all_finite.*")
def objective(trial):
    # --- A. Ponderación de Variables (Feature Weighting) ---

    # Mantenemos Geo fijo en 1.0 como referencia (ancla).
    w_geo = 1.0
    
    # Optuna probará qué tan importantes son los tiempos y las provincias
    w_log = trial.suggest_float("weight_logistics", 1.0, 3.0)   # Peso para tiempos
    w_cat = trial.suggest_float("weight_province", 0.5, 1.5)    # Peso para provincia (0 = ignorar)
    w_oth = trial.suggest_float("weight_other", 0.5, 1.5)       # Peso para Cota/Otros

    # Aplicar pesos (Trabajamos sobre una copia para no dañar el original)
    X_weighted = X_scaled.copy()
    
    # Multiplicamos las columnas por su peso
    # (Aumentar el peso = Aumentar la varianza = PCA le da más importancia)
    if geo_idxs: X_weighted[:, geo_idxs] *= w_geo
    if log_idxs: X_weighted[:, log_idxs] *= w_log
    if cat_idxs: X_weighted[:, cat_idxs] *= w_cat
    if other_idxs: X_weighted[:, other_idxs] *= w_oth

    # --- B. Reducción de Dimensionalidad (PCA) ---
    # Ajustamos n_components dinámicamente
    max_components = min(25, X.shape[1])
    n_components = trial.suggest_int("n_components", 5, max_components)
    
    pca = PCA(n_components=n_components, random_state=42)
    # IMPORTANTE: Pasamos la matriz pesada (X_weighted)
    X_pca = pca.fit_transform(X_weighted)

    # --- C. Clustering (HDBSCAN) ---
    min_cluster_size = trial.suggest_int("min_cluster_size", 10, 40)
    min_samples = trial.suggest_int("min_samples", 15, 35)

    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric='euclidean',
        cluster_selection_method='leaf',
        gen_min_span_tree=True 
    ).fit(X_pca)

    labels = clusterer.labels_
    unique_labels = set(labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)

    # --- Penalizaciones y Score ---
    if n_clusters < 30: return -1.0

    try:
        score = validity_index(X_pca, labels, metric='euclidean')
    except Exception:
        score = -1.0

    # Penalización por fragmentación excesiva
    if n_clusters > 72:
        score -= 0.02 * (n_clusters - 72)
        
    # Penalización extra si ignora demasiado la geografía (opcional)
    # Si w_log es muy alto, podría crear clusters que no tienen sentido geográfico
    if w_log > 2.5: 
        score -= 0.05

    return score

# --- Ejecutar Optimización ---
print("Iniciando optimización con Pesos Dinámicos...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n✅ ¡Optimización lista!")
print("Mejores pesos encontrados:")
print(f"  - Logística (Tiempos): {study.best_params.get('weight_logistics', 1.0):.2f}")
print(f"  - Provincias: {study.best_params.get('weight_province', 1.0):.2f}")

Iniciando optimización con Pesos Dinámicos...


[I 2025-11-19 16:31:56,020] A new study created in memory with name: no-name-97f98782-c1ff-4d1f-8afb-fd8d6c6fb106


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-19 16:31:57,335] Trial 0 finished with value: 0.254354787819289 and parameters: {'weight_logistics': 2.121473178818003, 'weight_province': 1.1809598003769672, 'weight_other': 1.0467800699805978, 'n_components': 17, 'min_cluster_size': 40, 'min_samples': 30}. Best is trial 0 with value: 0.254354787819289.
[I 2025-11-19 16:31:58,691] Trial 1 finished with value: 0.27424242067419907 and parameters: {'weight_logistics': 2.257846050294011, 'weight_province': 0.5777978936378615, 'weight_other': 0.95161381805983, 'n_components': 17, 'min_cluster_size': 18, 'min_samples': 28}. Best is trial 1 with value: 0.27424242067419907.
[I 2025-11-19 16:32:00,173] Trial 2 finished with value: 0.2306607966108466 and parameters: {'weight_logistics': 2.803727035479362, 'weight_province': 1.1929827936977957, 'weight_other': 1.0154910770810928, 'n_components': 14, 'min_cluster_size': 16, 'min_samples': 27}. Best is trial 1 with value: 0.27424242067419907.
[I 2025-11-19 16:32:02,539] Trial 3 finished

In [67]:
import pandas as pd
import numpy as np
import hdbscan
from sklearn.decomposition import PCA

# ==========================================
# 1. RECUPERAR PARÁMETROS Y PESOS
# ==========================================
best_params = study.best_params
print(f"Mejor Score: {study.best_value:.4f}")
print("Mejores parámetros:", best_params)

# Recuperamos los pesos ganadores (usamos .get(..., 1.0) por si alguno no existiera)
w_geo = 1  # El ancla fija que definimos
w_log = best_params.get("weight_logistics", 1.0)
w_cat = best_params.get("weight_province", 1.0)
w_oth = best_params.get("weight_other", 1.0)

# ==========================================
# 2. APLICAR LOS PESOS A LA MATRIZ X
# ==========================================
# Creamos una copia para no alterar la original
X_final_weighted = X_scaled.copy()

# Multiplicamos las columnas por sus pesos optimizados
# (Asumiendo que tienes las listas de índices: geo_idxs, log_idxs, etc. definidas)
if geo_idxs: X_final_weighted[:, geo_idxs] *= w_geo
if log_idxs: X_final_weighted[:, log_idxs] *= w_log
if cat_idxs: X_final_weighted[:, cat_idxs] *= w_cat
if other_idxs: X_final_weighted[:, other_idxs] *= w_oth

print("✅ Pesos aplicados a la matriz de datos.")

# ==========================================
# 3. PCA FINAL (Sobre la matriz pesada)
# ==========================================
pca_final = PCA(n_components=best_params["n_components"], random_state=42)
# ¡OJO! Aquí usamos X_final_weighted, NO X_scaled directo
X_pca_final = pca_final.fit_transform(X_final_weighted)

# ==========================================
# 4. HDBSCAN FINAL
# ==========================================
best_clusterer = hdbscan.HDBSCAN(
    min_cluster_size=best_params["min_cluster_size"],
    min_samples=best_params["min_samples"],
    metric='euclidean',
    cluster_selection_method='eom',
    gen_min_span_tree=True  # Recomendado dejarlo True por si quieres validar después
).fit(X_pca_final)

# ==========================================
# 5. ASIGNACIÓN DE ETIQUETAS
# ==========================================
# Inicializamos con -2 (valor seguro para "no procesado")
df['cluster_opt'] = -2

# Asignamos las etiquetas usando los índices válidos
# Asegúrate de que 'valid_indices' existe (viene del paso de limpieza de nulos)
df.loc[valid_indices, 'cluster_opt'] = best_clusterer.labels_

# Reporte
n_clusters_final = len(set(best_clusterer.labels_)) - (1 if -1 in best_clusterer.labels_ else 0)
print(f"\n🎯 Modelo Final Generado.")
print(f"   Clusters encontrados: {n_clusters_final}")
print(f"   Ruido (puntos sin cluster): {sum(best_clusterer.labels_ == -1)}")

Mejor Score: 0.2909
Mejores parámetros: {'weight_logistics': 1.4467689824235586, 'weight_province': 1.3439293360608395, 'weight_other': 0.6681052361132213, 'n_components': 23, 'min_cluster_size': 25, 'min_samples': 27}
✅ Pesos aplicados a la matriz de datos.

🎯 Modelo Final Generado.
   Clusters encontrados: 40
   Ruido (puntos sin cluster): 993


In [68]:
# 1. RECREAR valid_indices
# Hacemos exactamente el mismo filtro que cuando creamos la matriz X
# (Seleccionar numéricos -> Borrar Nulos -> Guardar el índice)
df_used_for_model = df_encoded.select_dtypes(include=[np.number]).dropna()
valid_indices = df_used_for_model.index

print(f"Índices recuperados: {len(valid_indices)} filas coinciden con el modelo.")

# 2. ASIGNAR ETIQUETAS (Ahora sí funcionará)
# Inicializamos con -2 (para identificar si alguna fila quedó fuera)
df['cluster_opt'] = -2

# Asignamos usando los índices. Pandas se encarga de emparejar cada ID con su cluster.
df.loc[valid_indices, 'cluster_opt'] = best_clusterer.labels_

# 3. REPORTE FINAL
n_clusters_final = len(set(best_clusterer.labels_)) - (1 if -1 in best_clusterer.labels_ else 0)
print(f"\n🎯 Asignación exitosa.")
print(f"   Clusters encontrados: {n_clusters_final}")
print(f"   Ruido: {sum(df['cluster_opt'] == -1)}")

Índices recuperados: 7908 filas coinciden con el modelo.

🎯 Asignación exitosa.
   Clusters encontrados: 40
   Ruido: 993


In [69]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

# ==========================================
# 1. PREPARAR COLORES ÚNICOS (Sin repetir)
# ==========================================
# Filtramos los datos (quitamos nulos y ruido para asignar colores)
plot_data = df[df['cluster_opt'] != -2].copy()
# Obtenemos la lista de clusters reales ordenados
clusters_reales = sorted(list(set(plot_data['cluster_opt']) - {-1}))
n_clusters = len(clusters_reales)

print(f"Generando paleta única para {n_clusters} clusters...")

# Usamos 'husl' de Seaborn: permite generar N colores visualmente distintos
# Es mucho mejor que 'tab20' cuando tienes muchos grupos.
palette = sns.color_palette("husl", n_clusters)
# Convertimos a un diccionario {ID_Cluster: Color}
color_map = {cluster: palette[i] for i, cluster in enumerate(clusters_reales)}
# Asignamos un gris claro para el ruido (-1)
color_map[-1] = (0.8, 0.8, 0.8) # Gris



# ==========================================
# 3. MAPA "CHILENO" REAL (Interactivo - Plotly)
# ==========================================
# Este es el mejor para ver "Chile" porque pone el mapa geográfico de fondo.
print("Generando mapa interactivo de Chile...")

# Convertimos la columna a string para que Plotly la trate como categoría (colores discretos)
plot_data['Cluster_String'] = plot_data['cluster_opt'].astype(str)

fig = px.scatter_mapbox(
    plot_data[plot_data['cluster_opt'] != -1], # Omitimos ruido para limpiar la vista
    lat="Latitud",
    lon="Longitud",
    color="Cluster_String",
    color_discrete_sequence=px.colors.qualitative.Dark24, # Paleta de alto contraste
    # Si tienes nombres de faena, descomenta la siguiente línea:
    # hover_name="NombreFaena", hover_data=["ProvinciaFaena"],
    zoom=4,
    center={"lat": -30.0, "lon": -71.0}, # Centrado aprox en Coquimbo/La Serena
    height=900,
    title="Mapa de Clusters Mineros (Fondo Geográfico)"
)

# Estilo del mapa (OpenStreetMap es gratuito y detallado)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_traces(marker=dict(size=8)) # Puntos un poco más grandes

fig.show()

Generando paleta única para 40 clusters...
Generando mapa interactivo de Chile...


In [70]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd

# --- CONFIGURACIÓN ---
# Umbral para unir centroides.
# 0.1 grados = ~11 km
# 0.2 grados = ~22 km (Si los centros de dos clusters están a <22km, se unen)
MERGE_THRESHOLD = 0.4

# Definimos explícitamente las columnas
col_origen = 'cluster_opt'   # La columna que generó Optuna
col_destino = 'cluster_merged' # La nueva columna con la fusión

print(f"--- Iniciando fusión basada en CENTROIDES (Umbral: {MERGE_THRESHOLD}°) ---")

# 1. Validación de seguridad
if col_origen not in df.columns:
    print(f"⚠️ Advertencia: '{col_origen}' no existe. Buscando alternativas...")
    if 'cluster_final' in df.columns: col_origen = 'cluster_final'
    elif 'cluster' in df.columns: col_origen = 'cluster'
    print(f"   -> Usando columna: '{col_origen}'")

# 2. Calcular el Centroide de cada cluster
# Ignoramos el ruido (-1) y los nulos (-2)
unique_labels = set(df[col_origen]) - {-1, -2}
centroids = []
label_map = []

for label in unique_labels:
    # Tomamos todas las minas de ESTE cluster específico
    cluster_data = df[df[col_origen] == label]
    points = cluster_data[['Latitud', 'Longitud']].values
    
    # Calculamos el punto medio geográfico (Centroide)
    centroid = points.mean(axis=0)
    
    centroids.append(centroid)
    label_map.append(label)

# 3. Aplicar la Unión (Clustering sobre Centroides)
if centroids:
    centroids = np.array(centroids)
    
    # AgglomerativeClustering agrupa los CENTROIDES que están cerca.
    # linkage='complete' garantiza que en el grupo resultante, 
    # todos los centroides disten menos de MERGE_THRESHOLD entre sí.
    agg = AgglomerativeClustering(
        n_clusters=None, 
        metric='euclidean', 
        linkage='complete', 
        distance_threshold=MERGE_THRESHOLD
    )
    
    # Obtenemos las nuevas etiquetas simplificadas
    merged_labels = agg.fit_predict(centroids)
    
    # 4. Mapear {Viejo -> Nuevo}
    merge_dict = {old: new for old, new in zip(label_map, merged_labels)}
    
    # El ruido y los nulos se quedan igual
    merge_dict[-1] = -1
    merge_dict[-2] = -2
    
    # 5. Aplicar al DataFrame
    df[col_destino] = df[col_origen].map(merge_dict)
    
    # Reporte
    n_before = len(unique_labels)
    n_after = len(set(merged_labels))
    print(f"✅ Fusión completada.")
    print(f"   Clusters Originales: {n_before}")
    print(f"   Clusters Fusionados: {n_after}")
    print(f"   Se unieron {n_before - n_after} clusters vecinos.")

else:
    print("⚠️ No se encontraron clusters válidos para fusionar.")

--- Iniciando fusión basada en CENTROIDES (Umbral: 0.4°) ---
✅ Fusión completada.
   Clusters Originales: 40
   Clusters Fusionados: 24
   Se unieron 16 clusters vecinos.


In [71]:
from hdbscan.validity import validity_index
from sklearn.metrics import silhouette_score
import numpy as np

print("--- Evaluando Calidad de los Clusters Finales ---")

# 1. PREPARAR DATOS
# Necesitamos las etiquetas y la matriz de datos original (X_pca_final o X_scaled)
# Asegúrate de usar la misma matriz con la que entrenaste el modelo.
if 'X_pca_final' in locals():
    data_for_score = X_pca_final
    print("   Usando datos PCA ponderados (X_pca_final).")
else:
    data_for_score = X_scaled
    print("   ⚠️ Variable 'X_pca_final' no encontrada. Usando 'X_scaled'.")

# Obtenemos las etiquetas finales (fusionadas)
# Es importante filtrar el ruido (-1) para el Silhouette, pero DBCV lo maneja.
labels_final = df.loc[valid_indices, 'cluster_merged'].values

# 2. CALCULAR DBCV (Density Based Clustering Validation)
# Este es el "dbscore" que usa Optuna.
try:
    dbcv_score = validity_index(data_for_score, labels_final, metric='euclidean')
    print(f"\n✅ DBCV Score Final: {dbcv_score:.4f}")
    print("   (Rango: -1 a 1. Mayor es mejor. Es normal que sea menor al de Optuna tras la fusión)")
except Exception as e:
    print(f"\n❌ No se pudo calcular DBCV: {e}")

# 3. CALCULAR SILHOUETTE SCORE (Opcional pero recomendado)
# Mide qué tan "separados" están los clusters visualmente.
# Silhouette ignora el ruido (-1) para no castigar injustamente.
mask_valid = labels_final != -1
if np.sum(mask_valid) > 0:
    sil_score = silhouette_score(data_for_score[mask_valid], labels_final[mask_valid])
    print(f"✅ Silhouette Score: {sil_score:.4f}")
    print("   (Rango: -1 a 1. Indica qué tan compactos y separados están los grupos)")
else:
    print("⚠️ No hay suficientes datos agrupados para calcular Silhouette.")

# 4. COMPARATIVA
print(f"\nResumen:")
print(f"   Clusters Totales: {len(set(labels_final) - {-1})}")
print(f"   Puntos de Ruido: {np.sum(labels_final == -1)}")

--- Evaluando Calidad de los Clusters Finales ---
   Usando datos PCA ponderados (X_pca_final).

✅ DBCV Score Final: 0.1904
   (Rango: -1 a 1. Mayor es mejor. Es normal que sea menor al de Optuna tras la fusión)
✅ Silhouette Score: 0.5106
   (Rango: -1 a 1. Indica qué tan compactos y separados están los grupos)

Resumen:
   Clusters Totales: 24
   Puntos de Ruido: 993


In [73]:
import plotly.express as px

# =============================================================================
# 5. VISUALIZACIÓN CON MAPA DE CHILE (PLOTLY)
# =============================================================================

# 1. Preparar los datos
# Filtramos los nulos (-2)
plot_data = df[df['cluster_merged'] != -2].copy()

# Convertimos la columna a texto (String) para que el mapa asigne 
# colores diferentes a cada grupo, en lugar de una barra de color numérica.
plot_data['Cluster_ID'] = plot_data['cluster_merged'].astype(str)

# Ordenamos para que la leyenda salga ordenada
plot_data = plot_data.sort_values('cluster_merged')

# 2. Generar el Mapa
fig = px.scatter_mapbox(
    plot_data,
    lat="Latitud",
    lon="Longitud",
    color="Cluster_ID",
    
    # Asignamos gris claro ('lightgray') específicamente al Ruido (-1)
    # y dejamos que Plotly asigne colores vibrantes al resto.
    color_discrete_map={'-1': 'lightgray'},
    
    # Datos que aparecerán al pasar el mouse sobre un punto
    hover_name="Cluster_ID", 
    hover_data=["ProvinciaFaena", "cluster_opt"], 
    
    # Configuración de la vista inicial (Norte Chico/Grande)
    zoom=4.5,
    center={"lat": -28.0, "lon": -70.5}, 
    
    height=900,
    title=f"Mapa de Zonas Logísticas (Fusión {MERGE_THRESHOLD}°)"
)

# 3. Estilo del Mapa
# "carto-positron" es un mapa limpio, ideal para tesis. 
# También puedes usar "open-street-map".
fig.update_layout(mapbox_style="carto-positron")

# Ajustar tamaño de los puntos y márgenes
fig.update_traces(marker=dict(size=9, opacity=0.8))
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout(legend_title_text='Zona Logística (ID)')

# Mostrar
fig.show()

# Opcional: Guardar como archivo HTML interactivo para la tesis
# fig.write_html("Mapa_Clusters_Fusionados.html")